In [3]:
import requests, re
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [4]:
base_url = 'https://www.hanbit.co.kr'
sub_url = '/store/books/new_book_list.html'
url = base_url + sub_url
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')


In [5]:
lis = soup.select('.sub_book_list')
len(lis)

20

In [6]:
li = lis[0]
href = li.find('a')['href']
href

'/store/books/look.php?p_code=B2831223974'

- sub page 에서 정보를 가져옴

In [7]:
res = requests.get(base_url + href)
book_soup = BeautifulSoup(res.text, 'html.parser')

In [8]:
info = book_soup.select_one('.store_product_info_box')
title = info.find('h3').string
title

'엑셀이 편해지는 파이썬'

In [9]:
book_lis = info.select('.info_list > li')
author = book_lis[0].find('span').get_text().strip()
author

'펠릭스 춤슈타인'

In [10]:
if len(book_lis) == 5:
    translator = '-'
else:
    translator = book_lis[1].find('span').get_text().strip()
translator

'한선용'

In [ ]:
date = book_lis[-4].find('span').get_text().strip()
date

In [ ]:
page = book_lis[-3].find('span').get_text().strip().split()[0]
page = int(page)
page

In [11]:
payments = book_soup.select_one('.payment_box.curr').find_all('p')
sales = payments[1].select_one('.pbr').get_text().split('(')[0]
sales = int(re.sub('[^0-9]','',sales))
sales

25200

In [ ]:
lines = []
for li in lis:
    book_url = li.find('a')['href']
    res = requests.get(base_url + book_url)
    book_soup = BeautifulSoup(res.text, 'html.parser')

    info = book_soup.select_one('.store_product_info_box')
    title = info.find('h3').string
    book_lis = info.select('.info_list > li')
    author = book_lis[0].find('span').get_text().strip()
    if len(book_lis) == 5:
        translator = '-'
    else:
        translator = book_lis[1].find('span').get_text().strip()
    date = book_lis[-4].find('span').get_text().strip()
    page = book_lis[-3].find('span').get_text().strip().split()[0]
    
    payments = book_soup.select_one('.payment_box.curr').find_all('p')
    sales = payments[1].select_one('.pbr').get_text().split('(')[0]
    sales = int(re.sub('[^0-9]','',sales))

    lines.append([title,author,translator,date,page,sales])

In [ ]:
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','페이지','판매가'])
df.tail()

In [ ]:
lines = []
for page in tqdm(range(1,11)):
    url = f'{base_url}{sub_url}?page={page}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    lis = soup.select('.sub_book_list')

    for li in lis:
        book_url = li.find('a')['href']
        res = requests.get(base_url + book_url)
        book_soup = BeautifulSoup(res.text, 'html.parser')

        info = book_soup.select_one('.store_product_info_box')
        title = info.find('h3').string
        book_lis = info.select('.info_list > li')
        author = book_lis[0].find('span').get_text().strip()
        if len(book_lis) == 5:
            translator = '-'
        else:
            translator = book_lis[1].find('span').get_text().strip()
        date = book_lis[-4].find('span').get_text().strip()
        page = book_lis[-3].find('span').get_text().strip().split()[0]
        
        payments = book_soup.select_one('.payment_box.curr').find_all('p')
        sales = payments[1].select_one('.pbr').get_text().split('(')[0]
        sales = int(re.sub('[^0-9]','',sales))

        lines.append([title,author,translator,date,page,sales])

In [ ]:
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','페이지','판매가'])
df.tail()

In [ ]:
df.to_csv('한빛신규도서.csv', index=False)